# menu:


Date:23/01/23

This file contains the functions required to create a graphical user interface using tkinter. The GUI, makes use of the additional scripts and acts as an easy to use controller for selecting data, producing tables and graphs. It also has additional features such as saving the graph and exporting tables as an Excel file. 

## Imports 

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} seaborn
!conda install --yes --prefix {sys.prefix} tabulate
import tabulate
import seaborn as sns
import sqlite3
from pathlib import Path
import os
import glob
import numpy as np
import pandas as pd
import ipynb
import threading
import matplotlib
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolours
import matplotlib.ticker as mtick
from tkinter import * 
import tkinter as tk
from tkinter import ttk
from tkinter import font as tkfont
from ipynb.fs.defs.CW_Preprocessing import *
from ipynb.fs.defs.Student_Att import *
from ipynb.fs.defs.Module_Att import *
from ipynb.fs.defs.Poor_Att import *
from IPython.display import clear_output



Retrieving notices: ...working... done
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



# GUI Pages

## Menu functions

In [2]:
class Menu(tk.Tk):
    """
    Menu class is used to establish the use of mltiple pages within tkinter via making seperate class types and looping
    them to stack one frame.
    """
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)

        self.title_font = tkfont.Font(family='Helvetica', size=18, weight="bold", slant="italic")
        self.body_font = tkfont.Font(family='Helvetica', size=10)
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        self.frames = {}
        for F in (HomePage, StudentAttPage, ModuleAttPage, PoorAttPage):
            page_name = F.__name__
            frame = F(parent=container, controller=self)
            self.frames[page_name] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame("HomePage")

    def show_frame(self, page_name):
        """Show a frame for the given page name"""
        frame = self.frames[page_name]
        frame.tkraise()
        frame.config(bg="#ADD8E6")

## Home Page functions

In [3]:
class HomePage(tk.Frame):
    """
    Home Page for the GUI, also used to establish colour, resizability and size of window.
    """
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        controller.geometry("854x480")
        controller.config(bg="#ADD8E6")
        label = tk.Label(self, text="Attendence Tracking System", font=controller.title_font, bg="#ADD8E6")
        label.place(relx=0.5, rely=0.3, anchor=CENTER)

        button1 = tk.Button(self, text="Check Student Attendence",
                            command=lambda: controller.show_frame("StudentAttPage"))
        button2 = tk.Button(self, text="Check Module Attendece",
                            command=lambda: controller.show_frame("ModuleAttPage"))
        button3 = tk.Button(self, text="Check Students with Poor Attendece",
                            command=lambda: controller.show_frame("PoorAttPage"))
        button1.place(relx=0.5, rely=0.5, anchor=CENTER)
        button2.place(relx=0.5, rely=0.6, anchor=CENTER)
        button3.place(relx=0.5, rely=0.7, anchor=CENTER)
        controller.resizable(False,False)
        

## Student Attendence Page functions

In [4]:
class StudentAttPage(tk.Frame):
    """
    Student attendence page, includes buttons which complete functions found in the StudentAtt class, which produce
    a table and bar chart when user inputs SID via text box and selects semester and variable via dropdown menu.
    """
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        title = tk.Label(self, text="Student Attendance", font=controller.title_font, bg="#ADD8E6")
        title.pack(side="top", fill="x", pady=10)
        label_sid = tk.Label(self, text="Please enter a SID:", font=controller.body_font, bg="#ADD8E6")
        label_sid.place(relx=0.344, rely=0.15, anchor=CENTER)
        label_semester = tk.Label(self, text="Please enter which semester:", font=controller.body_font, bg="#ADD8E6")
        label_semester.place(relx=0.315, rely=0.25, anchor=CENTER)
        label_type = tk.Label(self, text="Please pick a variable (changes both graph and table):",
                              font=controller.body_font, bg="#ADD8E6")
        label_type.place(relx=0.232, rely=0.35, anchor=CENTER)
        label_output = tk.Label(self, text="Please click an output:", font=controller.body_font, bg="#ADD8E6")
        label_output.place(relx=0.32, rely=0.45, anchor=CENTER)
        label_information = tk.Message(self, text="Please enter a single SID then choose a semester.\
                                       \nYou can then plot a table or graph using the inputted SID.\
                                       \nOptionally you may also change the variable which the graph and\
                                       table use via selecting a different variable (week is default).",
                                       font=controller.body_font, bg="#ADD8E6")
        label_information.place(relx=0.63, rely=0.12)
        button = tk.Button(self, text="Go back to the home page",
                           command=lambda: controller.show_frame("HomePage"))
        button.place(relx=0.5, rely=0.9, anchor=CENTER)
        clearButton = tk.Button(self,
                        text = "Clear output", 
                        command = clear_output)
        clearButton.place(relx=0.5, rely=0.8, anchor=CENTER)
        
        def printsid():
            """
            Verifies input then plots StudentAtt.sid_table with inputted SID, Semester and variable option.
            """
            try:
                sid = inputsid.get()
                semester = semester_value.get()
                sid = int(sid)
                semester = int(semester)
                sid_list = get_sid_list()
                semester_list = get_semester_list()
                variable = str(value_inside.get())
                if sid in sid_list:
                    if semester in semester_list:
                        StudentAtt.sid_table(sid,semester, variable)
                        plt.show()
                        sidlbl.config(text = 'Plotted student attendance table!')
                    else:
                        sidlbl.config(text = 'The inputed semester was not found')
                else:
                    sidlbl.config(text = 'The inputed SID was not found')
            except:
                sidlbl.config(text = 'Error with inputed SID or Semester')
                
                
                
        def printsid_graph():
            """
            Verifies input then plots StudentAtt.plot_sid_graph with inputted SID, Semester and variable option.
            """
            try:
                sid = inputsid.get()
                semester = semester_value.get()
                sid = int(sid)
                semester = int(semester)
                sid_list = get_sid_list()
                semester_list = get_semester_list()
                variable = str(value_inside.get())
                if sid in sid_list:
                    if semester in semester_list:
                        StudentAtt.plot_sid_graph(sid,semester, variable)
                        plt.show()
                        sidlbl.config(text = 'Plotted student attendance graph!')
                    else:
                        sidlbl.config(text = 'The inputed semester was not found')
                else:
                    sidlbl.config(text = 'The inputed SID was not found')
            except:
                sidlbl.config(text = 'Error with inputed SID or Semester')
                
                
        def sid_list():
            """
            Used to produce a list of SID's used during verification.
            """
            df = PoorAtt.all_student_att_week_table(1)
            df1 = PoorAtt.full_clean_table(df)
            df2 = df1.data
            df2 = df2['sid']
            df2 = df2.sort_values()
            with pd.option_context('display.max_rows', None):
                return print(df2.to_markdown())
            
        def savegraph():
            """
            Verifies input then saves StudentAtt.plot_sid_graph with inputted SID, Semester and variable option. 
            File name "graph.png" found in the same directory as executed.
            """
            try:
                sid = inputsid.get()
                semester = semester_value.get()
                sid = int(sid)
                semester = int(semester)
                sid_list = get_sid_list()
                semester_list = get_semester_list()
                variable = str(value_inside.get())
                if sid in sid_list:
                    if semester in semester_list:
                        StudentAtt.plot_sid_graph(sid,semester, variable)
                        plt.savefig('graph.png')
                        sidlbl.config(text = 'Saved graph to directory!')
                    else:
                        sidlbl.config(text = 'The inputed semester was not found')
                else:
                    sidlbl.config(text = 'The inputed SID was not found')
            except:
                sidlbl.config(text = 'Error with inputed SID or Semester')
                
                
        options = ["week","day","type","time"]
        value_inside = tk.StringVar(self)
        value_inside.set("Select a variable")
        dropdown_var = OptionMenu(self, value_inside, *options)
        dropdown_var.place(relx=0.5, rely=0.35, anchor=CENTER)
        
        inputsid = tk.Entry(self, width = 10)
        inputsid.place(relx=0.5, rely=0.15, anchor=CENTER)
        
        sid_listButton = tk.Button(self,
                        text = "Print list of SID's", 
                        command = sid_list)
        sid_listButton.place(relx=0.7, rely=0.8, anchor=CENTER)
        
        saveButton = tk.Button(self,
                        text = "Save Graph", 
                        command = savegraph)
        saveButton.place(relx=0.3, rely=0.8, anchor=CENTER)
        
        semester = set(get_semester_list())
        semester_value = tk.StringVar(self)
        semester_value.set("Select a Semester")
        
        dropdown_semester = OptionMenu(self, semester_value, *semester)
        dropdown_semester.place(relx=0.5, rely=0.25, anchor=CENTER)
        
        
        plotButton = tk.Button(self,
                        text = "Plot table", 
                        command = printsid)
        plotButton.place(relx=0.55, rely=0.45, anchor=CENTER)
        plot_graphButton = tk.Button(self,
                        text = "Plot graph", 
                        command = printsid_graph)
        plot_graphButton.place(relx=0.45, rely=0.45, anchor=CENTER)
        sidlbl = Label(self)
        sidlbl.place(relx=0.5, rely=0.52, anchor=CENTER)
        

## Module Attendence Page functions

In [5]:
class ModuleAttPage(tk.Frame):
    """
    Module attendence page, includes buttons which complete functions found in the ModuleAtt class, which produce
    a table when user inputs a module code and week via a dropdown menu.
    """
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        title = tk.Label(self, text="Module Attendance", font=controller.title_font, bg="#ADD8E6")
        title.pack(side="top", fill="x", pady=10)
        label_module = tk.Label(self, text="Please enter a Module code:", font=controller.body_font, bg="#ADD8E6")
        label_module.place(relx=0.32, rely=0.15, anchor=CENTER)
        label_week = tk.Label(self, text="Please enter which week:", font=controller.body_font, bg="#ADD8E6")
        label_week.place(relx=0.33, rely=0.25, anchor=CENTER)
        label_information = tk.Message(self, text="Please select a module then choose a week to be plotted.",
                                       font=controller.body_font, bg="#ADD8E6", width = 150)
        label_information.place(relx=0.63, rely=0.12)
        button = tk.Button(self, text="Go back to the start menu",
                           command=lambda: controller.show_frame("HomePage"))
        button.place(relx=0.5, rely=0.9, anchor=CENTER)
        clearButton = tk.Button(self,
                        text = "Clear output", 
                        command = clear_output)
        clearButton.place(relx=0.5, rely=0.8, anchor=CENTER)
        
        def printmodule():
            """
            Plots module_att_table with inputted Module code and week.
            """
            module = module_value.get()
            module = str(module)
            week = week_value.get()
            week = int(week)
            module_att_table(module, week)
            plt.show()
            sidlbl.config(text = 'Plotted module attendance table!')
        

        module = set(get_module_list())
        module_value = tk.StringVar(self)
        module_value.set("Select a Module")
        dropdown_module = OptionMenu(self, module_value, *module)
        dropdown_module.place(relx=0.5, rely=0.15, anchor=CENTER)
        
        
        week = set(get_week_list())
        week_value = tk.StringVar(self)
        week_value.set("Select a week")
        dropdown_week = OptionMenu(self, week_value, *week)
        dropdown_week.place(relx=0.5, rely=0.25, anchor=CENTER)
    
        plotButton = tk.Button(self,
                        text = "Plot table", 
                        command = printmodule)
        plotButton.place(relx=0.5, rely=0.35, anchor=CENTER)
        sidlbl = Label(self)
        sidlbl.place(relx=0.5, rely=0.52, anchor=CENTER)
        

## Poor Attendence Page functions

In [6]:

class PoorAttPage(tk.Frame):
    """
    Poor attendence page, includes buttons which complete functions found in the PoorAtt class, which produce
    a table and graph. The table is made from filtering down those students with attendence rates 20% below department
    average. While the graph is made via selecing SID's and the semester producing a stacked comparisson graph
    of the students average weekly attedence compared to the modules weekly average attendance.
    """
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        title = tk.Label(self, text="Highlighted Poor Attendance", font=controller.title_font, bg="#ADD8E6")
        title.pack(side="top", fill="x", pady=10)
        label_sid = tk.Label(self, text="Please enter a SID:", font=controller.body_font, bg="#ADD8E6")
        label_sid.place(relx=0.344, rely=0.15, anchor=CENTER)
        label_semester = tk.Label(self, text="Please enter which semester:", font=controller.body_font, bg="#ADD8E6")
        label_semester.place(relx=0.315, rely=0.25, anchor=CENTER)
        label_information = tk.Message(self, text="Select a semester then click 'Plot Comparison Graph',\
 to recieve a list of all students that are 20% lower than the department average.",
                                       font=controller.body_font, bg="#ADD8E6", width = 270)
        label_information.place(relx=0.65, rely=0.12)
        label_information2 = tk.Message(self, text= "You can then plot a graph using inputted SIDs\
                                       (seperate SID's with a comma to compare multiple students) \
                                       \n\nThis plots the students weekly attendance compared to the modules weekly averages.",
                                       font=controller.body_font, bg="#ADD8E6", width = 270)
        label_information2.place(relx=0.65, rely=0.3)
        button = tk.Button(self, text="Go back to the start menu",
                           command=lambda: controller.show_frame("HomePage"))
        button.place(relx=0.5, rely=0.9, anchor=CENTER)
        clearButton = tk.Button(self,
                        text = "Clear output", 
                        command = clear_output)
        clearButton.place(relx=0.5, rely=0.8, anchor=CENTER)
        
        def printpooratt():
            """
            Creates the PoorAtt.select_style_table and plots the table. 
            """
            try:
                semester = semester_value.get()
                semester = int(semester)
                student_att_week_table_df = PoorAtt.all_student_att_week_table(semester)
                clean_table_df = PoorAtt.full_clean_table(student_att_week_table_df)
                style_table_df = PoorAtt.select_style_table(clean_table_df)
                display(style_table_df)
                sidlbl.config(text = 'Plotted poor attendance Table!')
            except:
                sidlbl.config(text = 'Error: no semester selected')
        
        def printpooratt_graph():
            """
            Creates the PoorAtt.make_graphe and plots the graph, with the inputted SIDs and semester. 
            """
            try:
                semester_input = semester_value.get()
                semester_input = int(semester_input)
                if semester_input == None:
                    sidlbl.config(text = 'Error: No Semester Selected')
                else:
                    try:
                        sid_input = inputsid.get()
                        sid_input = tuple(map(int, sid_input.split(',')))
                        semester_list = get_semester_list()
                        PoorAtt.make_graph(*sid_input, semester = 1)
                        plt.show()
                        sidlbl.config(text = 'Plotted comparisson graph!')
                    except:
                        sidlbl.config(text = 'Error: Incorrect SID input')
            except:
                sidlbl.config(text = 'Error: No Semester Selected')
                
        def sid_list():
            df = PoorAtt.all_student_att_week_table(1)
            df1 = PoorAtt.full_clean_table(df)
            df2 = df1.data
            df2 = df2['sid']
            df2 = df2.sort_values()
            sidlbl.config(text = 'Printed list of SIDs')
            with pd.option_context('display.max_rows', None):
                return print(df2.to_markdown())
        
        def save_table_excel():
            """
            Creates the PoorAtt.select_style_table and exports the table to excel, saving the file in the directory as 
            "poor_attendance.xlsx"
            """
            try:
                semester = semester_value.get()
                semester = int(semester)
                df = PoorAtt.all_student_att_week_table(semester)
                df1 = PoorAtt.full_clean_table(df)
                df2 = PoorAtt.select_style_table(df1)
                df3 = df2.data
                df3.to_excel("poor_attendance.xlsx")
                sidlbl.config(text = 'Excel file saved to directory')
            except:
                sidlbl.config(text = 'Error: no semester selected')
        
        inputsid = tk.Entry(self, width = 10)
        inputsid.place(relx=0.5, rely=0.15, anchor=CENTER)
        
        
        
        
        semester = set(get_semester_list())
        semester_value = tk.StringVar(self)
        semester_value.set("Select a Semester")
        dropdown_semester = OptionMenu(self, semester_value, *semester)
        dropdown_semester.place(relx=0.5, rely=0.25, anchor=CENTER)
        
        
        plotButton = tk.Button(self,
                        text = "Plot Poor Attendence Table", 
                        command = printpooratt)
        plotButton.place(relx=0.5, rely=0.35, anchor=CENTER)
        plot_graphButton = tk.Button(self,
                        text = "Plot Comparison Graph", 
                        command = printpooratt_graph)
        plot_graphButton.place(relx=0.5, rely=0.45, anchor=CENTER)
        sidlbl = Label(self)
        sidlbl.place(relx=0.5, rely=0.52, anchor=CENTER)
        
        sid_listButton = tk.Button(self,
                        text = "Print list of SID's", 
                        command = sid_list)
        sid_listButton.place(relx=0.7, rely=0.8, anchor=CENTER)
        
        saveButton = tk.Button(self,
                        text = "Export Table", 
                        command = save_table_excel)
        saveButton.place(relx=0.3, rely=0.8, anchor=CENTER)
        

## Main code:

In [7]:

if __name__ == "__main__":
    app = Menu()
    app.mainloop()
